# IMPORT

In [1]:
# Importing the required libraries
import torch # Import the Pytorch library
import torchvision # Import the torchvision library
from torchvision import transforms # Import the transforms module from torchvision

from PIL import Image # Import the Image module from the Python Imaging Library (PIL)

import urllib # Import the urllib library for URL handling

In [2]:
# Load a pre-trained ResNet-18 model from the Pytorch hub
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

# The following lines are alternative ways to load different variants of the ResNet model
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

# Set the model to evaluation mode
model.eval()

Using cache found in /Users/fritt/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
# Download an example image from the pytorch website

# URL of the image to be downloaded
url = "https://github.com/pytorch/hub/raw/master/images/dog.jpg"

# Name to be given to the downloaded image file
filename = "local_files/dog.jpg"

# Try to download the image using the urllib.URLopener() method (for Python 2)
try:
    urllib.URLopener().retrieve(url, filename)

# If the above method fails, use the urllib.request.urlretrieve() method (for Python 3)
except:
    urllib.request.urlretrieve(url, filename)

In [4]:
# Import the required libraries
from PIL import Image
from torchvision import transforms

# Open the input image
input_image = Image.open(filename)

# Define the image pre-processing steps
preprocess = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256x256 pixels
    transforms.CenterCrop(224), # Crop the image to 224x224 pixels (removing any extra pixels)
    transforms.ToTensor(), # Convert the image to a Pytorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image using the pre-trained model's mean and standard deviation
])

# Apply the pre-processing steps to the image
input_tensor = preprocess(input_image)

# Add a batch dimension to the image (required by the model)
input_batch = input_tensor.unsqueeze(0)

# Move the input and model to GPU for faster processing if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# Disable gradient computation (not needed for inference)
with torch.no_grad():
    # Use the model to classify the image
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)


tensor([ 1.5915e-02, -1.5497e+00,  3.2030e-01, -2.0585e+00, -8.5747e-01,
         1.7843e+00,  1.4699e+00,  2.1626e+00,  4.4888e+00,  8.2885e-01,
        -5.7819e+00, -3.4969e+00, -4.0621e+00, -4.7517e+00, -3.8072e+00,
        -4.7243e+00, -1.2590e+00,  2.9813e-01, -2.0459e+00, -5.2885e-01,
        -3.5982e+00, -8.1424e-01, -2.7651e+00, -1.2770e+00, -3.4182e+00,
        -1.9048e+00, -3.0018e+00, -1.3471e+00, -1.8391e+00,  1.3937e+00,
        -2.0114e+00, -1.4137e+00, -2.3287e+00, -1.8197e+00, -1.1914e-01,
        -3.4102e+00, -1.6544e+00, -3.4496e+00, -2.6479e+00, -2.7407e+00,
        -2.2193e+00, -3.6509e+00, -4.1255e+00, -5.5946e+00, -1.7519e+00,
        -1.6900e+00, -9.8164e-01, -2.1251e+00, -3.5137e+00, -1.3320e+00,
        -1.1335e+00, -1.1564e+00, -2.2713e-02, -8.5797e-01, -1.2919e+00,
        -2.8682e+00,  6.6078e-01, -1.7178e+00, -1.2443e+00, -2.3362e+00,
        -5.7817e-02, -1.9204e+00, -2.5964e+00, -1.8020e+00, -1.5125e+00,
        -1.0843e+00, -4.0987e-01, -1.3090e+00, -9.4

In [ ]:
# Download ImageNet labels
#!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [5]:
# Read the categories from a text file
with open("local_files/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Get the top 5 categories with the highest probabilities
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Print the top 5 categories and their probabilities
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8846226930618286
Arctic fox 0.045805007219314575
white wolf 0.04427620768547058
Pomeranian 0.005621352232992649
Great Pyrenees 0.004652009811252356
